In [2]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import requests
from app.core.db import get_session, Book
from sqlmodel import select

# --------------------------
# 1. Helper functions
# --------------------------

def fetch_isbn(title: str, author: str) -> str | None:
    """Try to find ISBN using OpenLibrary search."""
    url = "https://openlibrary.org/search.json"
    params = {"title": title, "author": author}
    r = requests.get(url, params=params, timeout=10)
    if r.status_code != 200:
        return None
    docs = r.json().get("docs", [])
    if not docs:
        return None
    # Take first matching ISBN if available
    isbn_list = docs[0].get("isbn", [])
    return isbn_list[0] if isbn_list else None


def cover_from_isbn(isbn: str) -> str | None:
    """Build direct cover URL from ISBN."""
    if not isbn:
        return None
    return f"https://covers.openlibrary.org/b/isbn/{isbn}-L.jpg"


def cover_from_title_author(title: str, author: str) -> str | None:
    """Fallback cover lookup using title/author search."""
    url = "https://openlibrary.org/search.json"
    params = {"title": title, "author": author}
    r = requests.get(url, params=params, timeout=10)
    if r.status_code != 200:
        return None
    docs = r.json().get("docs", [])
    if not docs:
        return None
    cover_id = docs[0].get("cover_i")
    if cover_id:
        return f"https://covers.openlibrary.org/b/id/{cover_id}-L.jpg"
    return None


# --------------------------
# 2. Main population logic
# --------------------------

def populate_metadata():
    with get_session() as session:
        books = session.exec(select(Book)).all()
        for b in books:
            print(f"Processing: {b.title} by {b.author}")

            # Populate ISBN if missing
            if not b.isbn:
                b.isbn = fetch_isbn(b.title, b.author)
                if b.isbn:
                    print(f"  → Found ISBN: {b.isbn}")

            # Populate cover if missing
            if not b.cover_url:
                if b.isbn:
                    b.cover_url = cover_from_isbn(b.isbn)
                if not b.cover_url:
                    b.cover_url = cover_from_title_author(b.title, b.author)

                if b.cover_url:
                    print(f"  → Added cover URL")

            session.add(b)

        session.commit()
    print("✅ Metadata population complete.")

# --------------------------
# 3. Run it
# --------------------------

populate_metadata()


Processing: The Hobbit by J.R.R. Tolkien
  → Added cover URL
Processing: 1984 by George Orwell
  → Added cover URL
Processing: Pride and Prejudice by Jane Austen
  → Added cover URL
Processing: Dune by Frank Herbert
  → Added cover URL
✅ Metadata population complete.
